In [2]:
# 🛠️ 1. Installazione delle librerie (solo su Colab)
!pip install transformers accelerate pandas


In [12]:
# ⚙️ 2. Importazione dei moduli
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd
import os

# 🖥️ 3. Configurazione (evita problemi su alcune macchine)
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 🔍 4. Verifica della GPU
print("✅ GPU disponibile:", torch.cuda.is_available())

# 📦 5. Caricamento del modello Phi-3.5-mini-instruct
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=False
)

# 🔤 6. Caricamento del tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    trust_remote_code=True
)

# 🔁 7. Creazione della pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=20,
    temperature=0.2,
    top_p=1
)

# 📄 8. Caricamento del file CSV (caricalo manualmente in Colab prima)
from google.colab import files
uploaded = files.upload()  # Carica file_definizioni.csv

# Il CSV ha due colonne senza header: parola; definizione



✅ GPU disponibile: True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Saving file_definizioni.csv to file_definizioni.csv


In [16]:
# Caricamento CSV (modifica il percorso se serve)
df = pd.read_csv("file_definizioni.csv", sep=";", names=["ParolaChiave", "Definizione"])

etichette = []

for parola_chiave, gruppo in df.groupby("ParolaChiave"):
    # Concateno tutte le definizioni in un unico testo
    testo_completo = " ".join(gruppo["Definizione"].tolist())

    prompt = (
        f"Queste sono le definizioni della parola '{parola_chiave}':\n"
        f"{testo_completo}\n"
        "Suggerisci una singola parola che rappresenti il tema principale comune a tutte queste definizioni.\n"
        "Rispondi con una sola parola."
    )

    output = pipe(prompt)
    generated_text = output[0]["generated_text"]

    # Estraggo la risposta eliminando il prompt
    risposta = generated_text[len(prompt):].strip()
    parola_finale = risposta.split()[0] if risposta else ""

    etichette.append((parola_chiave, parola_finale))

# Stampa le etichette risultanti
print("\nEtichette generate per parola chiave:")
for k, v in etichette:
    print(f"{k}: {v}")

# Salvataggio su file
with open("etichette_per_parola.txt", "w", encoding="utf-8") as f:
    for k, v in etichette:
        f.write(f"{k}: {v}\n")


Etichette generate per parola chiave:
Ansia: Ansia
Bias cognitivo: Cognitivo
Pendrive: USB
Telecomando: Controllo
